In [ ]:
import pickle
import shapely
from pathlib import Path
import numpy as np
import pandas as pd
import xarray as xr
import rioxarray as rxa
import matplotlib.pyplot as plt

# Add main repo to path
import sys
from os.path import expanduser
sys.path.append(expanduser('../../'))

from spicy_snow.processing.snow_index import calc_delta_cross_ratio, calc_delta_gamma, \
    clip_delta_gamma_outlier, calc_snow_index, calc_snow_index_to_snow_depth
from spicy_snow.processing.wet_snow import id_newly_wet_snow, id_wet_negative_si, \
    id_newly_frozen_snow, flag_wet_snow

In [ ]:
for fp in Path('/Users/zachkeskinen/Documents/spicy-snow/scripts/optimize/rmse_insitu').glob('*no_flag.nc'):
    fp_lidar = next(Path('/Users/zachkeskinen/Documents/spicy-snow/scripts/optimize/rmse_out').glob(fp.name))
    da = xr.open_dataarray(fp)
    da2 = xr.open_dataarray(fp_lidar) * 100
    insitu_best = da.where(da==da.min(), drop=True).squeeze()
    lidar_best = da2.where(da2==da2.min(), drop=True).squeeze()

In [ ]:
snotel = pd.read_csv('637_STAND_WATERYEAR=2021.csv', skiprows= 3, parse_dates=['Date'])
snotel.loc[:, 'depth'] = snotel.loc[:, 'SNWD.I-1 (in) ']*0.0254
snotel_x = -115.6666
snotel_y = 43.93333

a,b,c = insitu_best['A'], insitu_best['B'], insitu_best['C']

ds_ = xr.open_dataset('/Users/zachkeskinen/Documents/spicy-snow/Lidar_s1_stacks/Mores_2021-03-15.nc').load()
dataset = ds_[['s1','deltaVV','ims','fcf','lidar-sd']]
ds = calc_delta_cross_ratio(dataset, A = a)
ds = calc_delta_gamma(ds, B = b, inplace=False)
ds = clip_delta_gamma_outlier(ds)
ds = calc_snow_index(ds)
ds = id_newly_wet_snow(ds)
ds = id_wet_negative_si(ds)
ds = id_newly_frozen_snow(ds)
ds = flag_wet_snow(ds)
ds = calc_snow_index_to_snow_depth(ds, C = c)

snotel['spicy'] = np.nan
snotel['wet'] = np.nan
# tolerance around each site 500 m\n",
tol = 0.00090009*2

for ts in ds.time:
    spicy_depth = ds['snow_depth'].sel(time = ts).sel(x = slice(snotel_x - tol, snotel_x + tol), y = slice(snotel_y + tol, snotel_y - tol)).mean()
    # snotel.loc[snotel.Date.dt.date == pd.to_datetime(ts.data).date(), 'wet'] = ds['wet_snow'].sel(time = ts).sel(x = snotel_x, y = snotel_y, method = 'nearest')
    snotel.loc[snotel.Date.dt.date == pd.to_datetime(ts.data).date(), 'spicy'] = spicy_depth

fig,ax = plt.subplots(figsize = (12,8))
snotel.dropna(subset = ['spicy'])[['depth','spicy','Date']].set_index('Date').plot(ax = ax, linestyle = '-')

# Now for lidar on same axes

a,b,c = lidar_best['A'], lidar_best['B'], lidar_best['C']

ds_ = xr.open_dataset('/Users/zachkeskinen/Documents/spicy-snow/Lidar_s1_stacks/Mores_2021-03-15.nc').load()
dataset = ds_[['s1','deltaVV','ims','fcf','lidar-sd']]
ds = calc_delta_cross_ratio(dataset, A = a)
ds = calc_delta_gamma(ds, B = b, inplace=False)
ds = clip_delta_gamma_outlier(ds)
ds = calc_snow_index(ds)
ds = id_newly_wet_snow(ds)
ds = id_wet_negative_si(ds)
ds = id_newly_frozen_snow(ds)
ds = flag_wet_snow(ds)
ds = calc_snow_index_to_snow_depth(ds, C = c)

snotel['spicy'] = np.nan
snotel['wet'] = np.nan
# tolerance around each site 500 m\n",
tol = 0.00090009*2

for ts in ds.time:
    spicy_depth = ds['snow_depth'].sel(time = ts).sel(x = slice(snotel_x - tol, snotel_x + tol), y = slice(snotel_y + tol, snotel_y - tol)).mean()
    # snotel.loc[snotel.Date.dt.date == pd.to_datetime(ts.data).date(), 'wet'] = ds['wet_snow'].sel(time = ts).sel(x = snotel_x, y = snotel_y, method = 'nearest')
    snotel.loc[snotel.Date.dt.date == pd.to_datetime(ts.data).date(), 'spicy'] = spicy_depth

snotel.dropna(subset = ['spicy'])[['spicy','Date']].set_index('Date').plot(ax = ax, linestyle = '-')
ax.legend(['Snotel Snow Depth', 'Spicy (In-Situ Params)', 'Spicy (Lidar Params)'])
plt.title('Mores Creek Snotel vs Spicy Snow Depth (200 m radius)')

In [ ]:
snotel = pd.read_csv('637_STAND_WATERYEAR=2021.csv', skiprows= 3, parse_dates=['Date'])
snotel.loc[:, 'depth'] = snotel.loc[:, 'SNWD.I-1 (in) ']*0.0254
snotel_x = -115.6666
snotel_y = 43.93333

a,b,c = insitu_best['A'], insitu_best['B'], insitu_best['C']

ds_ = xr.open_dataset('/Users/zachkeskinen/Documents/spicy-snow/Lidar_s1_stacks/Mores_2021-03-15.nc').load()
dataset = ds_[['s1','deltaVV','ims','fcf','lidar-sd']]
ds = calc_delta_cross_ratio(dataset, A = a)
ds = calc_delta_gamma(ds, B = b, inplace=False)
ds = clip_delta_gamma_outlier(ds)
ds = calc_snow_index(ds)
ds = id_newly_wet_snow(ds)
ds = id_wet_negative_si(ds)
ds = id_newly_frozen_snow(ds)
ds = flag_wet_snow(ds)
ds = calc_snow_index_to_snow_depth(ds, C = c)

snotel['spicy'] = np.nan
snotel['wet'] = np.nan
# tolerance around each site 500 m\n",
tol = 0.00090009*2

for ts in ds.time:
    spicy_depth = ds['snow_depth'].sel(time = ts).sel(x = slice(snotel_x - tol, snotel_x + tol), y = slice(snotel_y + tol, snotel_y - tol)).mean()
    # snotel.loc[snotel.Date.dt.date == pd.to_datetime(ts.data).date(), 'wet'] = ds['wet_snow'].sel(time = ts).sel(x = snotel_x, y = snotel_y, method = 'nearest')
    snotel.loc[snotel.Date.dt.date == pd.to_datetime(ts.data).date(), 'spicy'] = spicy_depth

fig,ax = plt.subplots(figsize = (12,8))
snotel.dropna(subset = ['spicy'])[['depth','spicy','Date']].set_index('Date').plot(ax = ax, linestyle = '-')

# Now for lidar on same axes

a,b,c = lidar_best['A'], lidar_best['B'], lidar_best['C']

ds_ = xr.open_dataset('/Users/zachkeskinen/Documents/spicy-snow/Lidar_s1_stacks/Mores_2021-03-15.nc').load()
dataset = ds_[['s1','deltaVV','ims','fcf','lidar-sd']]
ds = calc_delta_cross_ratio(dataset, A = a)
ds = calc_delta_gamma(ds, B = b, inplace=False)
ds = clip_delta_gamma_outlier(ds)
ds = calc_snow_index(ds)
ds = id_newly_wet_snow(ds)
ds = id_wet_negative_si(ds)
ds = id_newly_frozen_snow(ds)
ds = flag_wet_snow(ds)
ds = calc_snow_index_to_snow_depth(ds, C = c)

snotel['spicy'] = np.nan
snotel['wet'] = np.nan
# tolerance around each site 500 m\n",
tol = 0.00090009*2

for ts in ds.time:
    spicy_depth = ds['snow_depth'].sel(time = ts).sel(x = slice(snotel_x - tol, snotel_x + tol), y = slice(snotel_y + tol, snotel_y - tol)).mean()
    # snotel.loc[snotel.Date.dt.date == pd.to_datetime(ts.data).date(), 'wet'] = ds['wet_snow'].sel(time = ts).sel(x = snotel_x, y = snotel_y, method = 'nearest')
    snotel.loc[snotel.Date.dt.date == pd.to_datetime(ts.data).date(), 'spicy'] = spicy_depth

snotel.dropna(subset = ['spicy'])[['spicy','Date']].set_index('Date').plot(ax = ax, linestyle = '-')
ax.legend(['Snotel Snow Depth', 'Spicy (In-Situ Params)', 'Spicy (Lidar Params)'])
plt.title('Mores Creek Snotel vs Spicy Snow Depth (200 m radius)')